# Multi-Camera Person Analysis
This notebook demonstrates how to use the CCTV analysis package for:
1. Person detection using YOLOv8x6
2. Person re-identification across cameras
3. Demographic analysis
4. Cross-camera tracking and matching

In [ ]:
import numpy as np
import datetime
import cv2
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.append(str(project_root / 'src'))

from cctv_analysis.matcher import PersonMatcher
from cctv_analysis.demographics import DemographicAnalyzer
from cctv_analysis.reid import PersonReID
from cctv_analysis.detector import PersonDetector

# Configure logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def create_id_generator():
    """Creates an infinite ID generator"""
    def infinite_counter():
        counter = 0
        while True:
            yield counter
            counter += 1
    return infinite_counter()

class VideoProcessor:
    def __init__(self, device="cuda"):
        """Initialize all models and components"""
        self.device = device
        self.detector = PersonDetector(device=device)
        self.reid_model = PersonReID(
            model_path=str(project_root / "models/reid/osnet_x1_0.pth"),
            device=device
        )
        self.demographic_analyzer = DemographicAnalyzer(device=device)
        self.matcher = PersonMatcher(similarity_threshold=0.5, max_time_diff=3600)
        self.person_id_counter = create_id_generator()
        
    def process_frame(self, frame, camera_id, timestamp):
        """Process a single frame"""
        if frame is None:
            return []

        # Detect persons
        detections = self.detector.detect(frame, conf_thresh=0.3)

        results = []
        for det in detections:
            try:
                x1, y1, x2, y2, conf = map(int, det)

                # Ensure valid coordinates
                x1, x2 = max(0, x1), min(frame.shape[1], x2)
                y1, y2 = max(0, y1), min(frame.shape[0], y2)

                # Skip if bbox is too small
                if (x2 - x1) < 20 or (y2 - y1) < 20:
                    continue

                # Extract person crop
                person_crop = frame[y1:y2, x1:x2].copy()

                # Extract ReID features
                reid_features = self.reid_model.extract_features(person_crop)

                if reid_features is not None:
                    # Analyze demographics
                    demographics = self.demographic_analyzer.analyze(person_crop)

                    # Generate unique person ID
                    person_id = next(self.person_id_counter)

                    # Add to matcher
                    self.matcher.add_person(
                        camera_id=camera_id,
                        person_id=person_id,
                        timestamp=timestamp,
                        features=reid_features,
                        demographics=demographics[0] if demographics else None
                    )

                    results.append({
                        'id': person_id,
                        'bbox': (x1, y1, x2, y2),
                        'conf': float(conf),
                        'demographics': demographics[0] if demographics else None
                    })

            except Exception as e:
                logging.error(f"Error processing detection: {e}")
                continue

        return results

    def process_video(self, video_path, camera_id, start_time, display=True, skip_frames=5):
        """Process entire video"""
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = 0

        logging.info(f"Processing video {video_path}")
        logging.info(f"Total frames: {total_frames}, FPS: {fps}")

        # Create progress bar
        pbar = tqdm(total=total_frames // skip_frames, 
                    desc=f"Camera {camera_id}")

        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Process every nth frame
                if frame_count % skip_frames == 0:
                    # Calculate timestamp
                    timestamp = start_time + datetime.timedelta(seconds=frame_count/fps)

                    # Process frame
                    results = self.process_frame(frame, camera_id, timestamp)

                    # Visualize results
                    if display and results:
                        frame = self.draw_results(frame, results)
                        cv2.imshow(f'Camera {camera_id}', frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break

                    pbar.update(1)

                frame_count += 1

        except Exception as e:
            logging.error(f"Error processing video: {e}")
            raise

        finally:
            cap.release()
            pbar.close()

    def draw_results(self, frame, results):
        """Draw detection results on frame"""
        for person in results:
            x1, y1, x2, y2 = person['bbox']
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Prepare label text
            label = f"ID: {person['id']}"
            if person['demographics']:
                demo = person['demographics']
                label += f" | {demo['gender']} | {demo['age_group']}"

            # Draw label
            cv2.putText(frame, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return frame

    def analyze_results(self):
        """Print analysis results"""
        self.matcher.print_matching_stats()
        self.matcher.visualize_matches(n_samples=5)

In [ ]:
# Initialize video processor
processor = VideoProcessor(device="cuda")

# Define start times for both videos
start_time_1 = datetime.datetime(2024, 1, 1, 12, 0, 0)
start_time_2 = datetime.datetime(2024, 1, 1, 12, 0, 30)  # 30 seconds later

# Process videos
try:
    processor.process_video(
        video_path=project_root / 'data/processed_data/D04_20241108.mp4',
        camera_id=2,
        start_time=start_time_1
    )
    
    processor.process_video(
        video_path=project_root / 'data/processed_data/D10_20241108.mp4',
        camera_id=1,
        start_time=start_time_2
    )
    
finally:
    cv2.destroyAllWindows()

In [ ]:
# Analyze results
processor.analyze_results()